In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
import numpy as np
import string
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [3]:
import glob
import os
asr = []
for filename in glob.glob('/Volumes/SHAH/thesis/E2E/*.txt'):
    with open(filename, 'r') as f:
        x = f.readline()
        if x == "":
            continue
        asr.append((filename.split("/")[5], x))

In [4]:
asr_df = pd.DataFrame(asr, columns=['File Name', 'Sentence'])

In [5]:
asr_df.head(10)

,File Name,Sentence
0,DavidPogue_2008P_71.txt,photocopier
1,Diego-20131023-zpg_a0169.txt,stop for an instant
2,anonymous-20090122-aka_b0041.txt,but if you're did not return until tomorrow
3,cyteen-20130524-prw_a0250.txt,he said the business license Hawaii and develop
4,sample-108342.txt,around the corner
5,3092-39501-0009.txt,Scotland the 4th of the blows of the heaviest ...
6,6308-68359-0031.txt,find a trace of his body there must have been ...
7,JohnDelaney_2010Z_58.txt,information I need about everything
8,8643-269254-0009.txt,no attempt was made to compare 40 version with...
9,6281-65531-0002.txt,completely


In [6]:
len(asr_df)

8589

In [7]:
import glob
import os
original = []
for filename in glob.glob('/Volumes/SHAH/thesis/txt/*.txt'):
    with open(filename, 'r') as f:
        x = f.readline()
        if x == "":
            continue
        original.append((filename.split("/")[5], x))

In [8]:
original_df = pd.DataFrame(original, columns=['File Name', 'Sentence'])

In [9]:
original_df.head()

,File Name,Sentence
0,anonymous-20100809-adw_b0364.txt,IN THE BOHEMIAN CLUB OF $SAN FRANCISCO] THERE ...
1,Fandark-20100822-acy_b0007.txt,HEAR THE INDIAN DOGS WAILING DOWN AT $CHURCHILL]
2,7912-105669-0020.txt,HAD CORRECTED ONE CONSIDERABLE PART OF HIS EAR...
3,3063-138652-0046.txt,THEY LOOKED UP THE HOOK BY WHICH |BOB] HAD HEL...
4,sample-161064.txt,I'M AFRAID THE JOKE'S ON $US]


In [10]:
original_dataset = []
for file_name, sentence in original_df.values.tolist():
    if file_name in asr_df['File Name'].values.tolist():
        original_dataset.append((file_name, sentence))

In [11]:
new_original_df = pd.DataFrame(original_dataset, columns=['File Name', 'Sentence'])

In [12]:
new_original_df.head()

,File Name,Sentence
0,knotyouraveragejo-20080426-adv_adv0186.txt,$BORON] AND $SILICON] ANOTHER POTASSIUM SODIUM...
1,uklebot-20090813-zxl_a0238.txt,IT HAPPENED TO HIM AT THE GALLINA SOCIETY IN $...
2,DanijelK-20100117-pbj_a0407.txt,{MERCEDES] SCREAMED CRIED LAUGHED AND MANIFEST...
3,3179-7153-0044.txt,|DIANA] AND |RUBY GILLIS] AND |JANE ANDREWS] A...
4,CKWilliams_2001_52.txt,MY STARTING TO FANCY SHE'D ENDED UP IN THIS FI...


In [13]:
asr_dataset = []
for file_name, sentence in asr_df.values.tolist():
    if file_name in new_original_df['File Name'].values.tolist():
        asr_dataset.append((file_name, sentence))

In [14]:
len(asr_dataset)

2494

In [15]:
new_asr_df = pd.DataFrame(asr_dataset, columns=['File Name', 'Sentence'])

In [16]:
new_asr_df.head()

,File Name,Sentence
0,DavidPogue_2008P_71.txt,photocopier
1,Diego-20131023-zpg_a0169.txt,stop for an instant
2,anonymous-20090122-aka_b0041.txt,but if you're did not return until tomorrow
3,cyteen-20130524-prw_a0250.txt,he said the business license Hawaii and develop
4,sample-108342.txt,around the corner


In [23]:
new_asr_df.sort_values('File Name', ignore_index=True, inplace=True)
new_asr_df['Sentence #'] = new_asr_df.index

In [24]:
new_original_df.sort_values('File Name', ignore_index=True, inplace=True)
new_original_df['Sentence #'] = new_original_df.index

In [21]:
new_asr_df.to_csv("../../data/e2e_asr_preprocessed.csv",index=False)

In [22]:
new_original_df.to_csv("../../data/e2e_original_preprocessed.csv",index=False)

In [26]:
new_original_df['Sentence'].values.tolist()[0]

'THE FLASHING AND GOLDEN PAGEANT OF $CALIFORNIA] THE SUDDEN AND GORGEOUS DRAMA THE SUNNY AND AMPLE LANDS THE LONG AND VARIED STRETCH FROM $PUGET SOUND] TO $COLORADO SOUTH]'

In [ ]:
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [51]:
labels = []
tokens = []
for sentence in new_original_df['Sentence'].values.tolist():
    x = []
    y = []
    #print(find_between_r(sentence, "$", "]"))
    
    break

COLORADO SOUTH


In [74]:
labels = []
tokens = []
dataset = []
sentence_no = 0
for sentence in new_original_df['Sentence'].values.tolist():
    x = []
    y = []
    loc = False
    org = False
    per = False
    for word in sentence.split():
        if "$" in word:
            if "]" in word:
                x.append(word[1:len(word)-1])
                y.append("LOC")
            else:
                x.append(word[1:len(word)])
                y.append("LOC")
                loc = True
                continue
        elif "{" in word:
            if "]" in word:
                x.append(word[1:len(word)-1])
                y.append("ORG")
            else:
                x.append(word[1:len(word)])
                y.append("ORG")
                org = True
                continue
        elif "|" in word:
            if "]" in word:
                x.append(word[1:len(word)-1])
                y.append("PER")
            else:
                x.append(word[1:len(word)])
                y.append("PER")
                per = True
                continue
        else:
            if loc:
                x.append(word[0:len(word) - 1])
                y.append("LOC")
                loc = False
                continue
            if org:
                x.append(word[0:len(word) - 1])
                y.append("ORG")
                org = False
                continue
            if per:
                x.append(word[0:len(word) - 1])
                y.append("PER")
                per = False
                continue
            x.append(word)
            y.append("O")
    for token, label in zip(x,y):
        dataset.append((sentence_no, token, label))
    tokens.append(" ".join(x))
    labels.append(",".join(y))
    sentence_no = sentence_no + 1

In [76]:
new_df = pd.DataFrame(dataset, columns=['Sentence #', 'Word', 'Tag'])
new_df.head()

,Sentence #,Word,Tag
0,0,THE,O
1,0,FLASHING,O
2,0,AND,O
3,0,GOLDEN,O
4,0,PAGEANT,O


In [77]:
new_df.to_csv("../../data/e2e_original_preprocessed_with_labels.csv",index=False)